In this tutorial we are going to implement Grover's quantum algorithm for unstructured search.

## <center>Introduction</center>  

Grover's algorithm is often described in the context of _searching an unstructured database_. One can generalize the search problem to the related problem of function inversion. Consider a function, $f(x)$, which can take on the value $0$ or $1$, depending on the argument $x$. For example, for $x = 10$, the function $f(10) = 1$, and it's $0$ for all other $x$ values.  



Now, the function inversion problem is the following. Given $f(x) = 1$, can we find the value of $x$ or a value of $x$ if there is more than one solution.  

<img src='images/grover2.png' width=700 >  

Classically, one has little choice but to try different x-values and test the function. So for $N$ total $x$-values of which $M$ satisfy $f(x) = 1$, it will take on the order of $N/M$ tries to succeed with high probability.
However, on a quantum computer, the same task succeeds with high probability in only square root of $N$ over $M$ tries, a quadratic improvement over classical computing. And while a quadratic improvement may not seem like a lot, it does become practically significant for problems with large $N$.  

To implement Grover's algorithm, we need an operational block that can recognize if a test value of $x$ satisfies our function or not.  
This operation is called an oracle. And it's a key part of each trial, which we'll call a Grover iteration.  

Now, it may seem challenging or even cheating to define an oracle that represents, for example, our database.
**After all, in loading such a database into our quantum computer, haven't we already searched it?**
That's certainly true, but this isn't how an oracle works.  

Again, it's worthwhile to think in terms of a mathematical function. The oracle doesn't need to know in advance every answer to every function call, nor does it need to have a ledger of all the input and output values.  
Rather, it just needs to be able to calculate the function and recognize when a particular condition is satisfied.  

And conceptually, we can program different functions into our oracle while keeping the underlying machinery in place. (_We won't discuss in detail how this is done, as it's obviously problem-dependent._)  
But we just note that this type of oracle can be designed using **reversible logic approaches.**  

The idea behind Grover's algorithm is the following. We start with an index register of values $x$ running from $[0, N - 1]$.  
The oracle takes a test value $x$, calculates the function $f(x)$, and performs modulo-2 addition of $f(x)$ with a helper qubit, which we'll call the oracle qubit. The oracle qubit is prepared in a superposition state  
$$\lvert q\rangle = \frac{\lvert 0\rangle - \lvert 1\rangle}{\sqrt{2}}.$$  

<center> **The Oracle** </center>  

$$\lvert x\rangle_{index ~ register} \lvert q\rangle_{oracle ~ qubit} \rightarrow \lvert x\rangle \lvert q \oplus f(x)\rangle$$  


$$
\begin{array}{cc} 
f(x)=0: no ~ change: & \lvert q\rangle \rightarrow \frac{\lvert 0 \oplus 0 \rangle - \lvert 1 \oplus 0\rangle}{\sqrt{2}} = \frac{\lvert 0\rangle - \lvert 1\rangle}{\sqrt{2}} ,\\ 
f(x)=1: pick ~ up ~ -1: & \lvert q\rangle \rightarrow \frac{\lvert 0 \oplus 1 \rangle - \lvert 1 \oplus 1\rangle}{\sqrt{2}} =  -\frac{\lvert 0\rangle - \lvert 1\rangle}{\sqrt{2}}.\end{array} \Bigg\} 
(-1)^{f(x)}\frac{\lvert 0\rangle - \lvert 1\rangle}{\sqrt{2}}
$$  


Since there are $N$ possible values of $x$, we'll choose $n = \log N$ qubits for register $1$. And register $2$ will hold a single qubit, which is the oracle qubit. Register $1$ qubits are prepared in the equal superposition state $\lvert\psi\rangle=\frac{1}{\sqrt{N}}\sum_{x=0}^{N-1}\lvert x\rangle$ and the oracle qubit in state $\frac{\lvert 0\rangle - \lvert 1\rangle}{\sqrt{2}}$.  

The next step is to implement a Grover iteration on the order of $\sqrt{N}$ times to obtain a solution with order unit probability.  Each iteration contains the oracle operator $O$, which with the help of the oracle qubit, will leave a $-1$ on the register $1$ states for values $x$ which satisfy $f(x) = 1$. It's a reflection operation. The remainder of the Grover iteration is a conditional phase shift, nestled between Hadamard gates, which leaves state $\lvert 0\rangle$ alone and, otherwise, places a $-1$ phase shift in front of all other states $\lvert x\rangle$. This can be written in terms of an equal superposition state, just like the one we started with. 

$$W = H^{\otimes n}(2\lvert 0\rangle \langle 0 \rvert - I)H^{\otimes n} = 2\lvert \psi\rangle \langle \psi \rvert - I.$$  

This set of three operations can also be viewed as a reflection, this time, across the vector $\lvert\psi\rangle$.
Together with the oracle, these two reflections in the Grover iteration serve to rotate the quantum state by an angle $\theta$ (see below for geometrical interpretation).  

**To sum up**, Grover's algorithm can be decomposed into three steps: 
1. Initialization of the system, 
1. an Oracle operator $O$, and
1. a (second) reflection operator $W$. 

For a list of $N = 2^n$ items, the system is initialized in the $n = \log N$-qubit equal superposition state. The Oracle operator encodes and marks the information in the list onto the quantum system; this can be viewed as the first reflection. Once the list is encoded, the operator $W$ applies a second reflection. The resulting state after these two operations is closer to the solution state corresponding to the marked item. The product of the two reflections $G = WO$ -- the Grover iteration -- is applied several times, and the number of repetitions necessary to find the marked item is upper bounded by $\frac{\pi}{4}\sqrt{N}$.

Now, we will assume that there is only one marked item, which we will call $\beta$. We start by defining a function f(x) such that the function is true, $f(x)=1$, only if the input x corresponds to the marked item $\beta$, and zero otherwise. Mathematically, this can be written as:  

 $$f\left( x\right) = \Bigg\{
 \begin{array}{ccc} 0 & \text {if} & x\neq \beta ,\\ 1 & \text {if} & x=\beta.\end{array}$$  

For a list of $N=2^n$ items, Grover's algorithm requires an n-qubit quantum system. We can associate each one
of the $N$ items of the list, $x$, with one of states of the $n$-qubit computational basis $x \rightarrow \lvert x\rangle$.  

Let's consider the following associations:

* For a list of $N=4$  items and a system of  two-qubits 

|Item in decimal|	Item in binary|	Basis state|
|---|---|---|
|0| 	 00 	 |$\lvert 00\rangle$| 
|1| 	 01 	 |$\lvert 01\rangle$| 
|2| 	 10 	 |$\lvert 10\rangle$|
|3| 	 11 	 |$\lvert 11\rangle$|  

* For a list of $N=8$  items and a system of three-qubits  

|Item in decimal|	Item in binary|	Basis state|
|---|---|---|
|0| 	 000 	 |$\lvert 000\rangle$|
|1| 	 001 	 |$\lvert 001\rangle$|
|2| 	 010 	 |$\lvert 010\rangle$|
|3| 	 011 	 |$\lvert 011\rangle$|
|4| 	 100 	 |$\lvert 100\rangle$|
|5| 	 101 	 |$\lvert 101\rangle$|
|6| 	 110 	 |$\lvert 110\rangle$|
|7| 	 111 	 |$\lvert 111\rangle$|  

To encode the function $f(x)$ into a quantum computer, the Oracle operator $O$ is applied onto the equal $n$-qubit superposition state. This operation acts as an identity for each basis state that does not correspond to the marked item, and adds a negative sign to the solution state $\lvert \beta \rangle$,  

$$\left\vert x\right\rangle \rightarrow \left( -1\right) ^{f\left( x\right) }\left\vert x\right\rangle =\left\{ \begin{array}{ccc} \left\vert x\right\rangle & \text {if} & x\neq \beta, \\ -\left\vert x\right\rangle & \text {if} & x=\beta.\end{array}\right.$$  

* **𝑁=4  example:** If the marked item is $\beta=2$ in decimal, then the function $f$ is given by

$$f\left( x\right) =\left\{ \begin{array}{lll} 0 & \text {if} & x=00,01,11 ,\\ 1 & \text {if} & x=10.\end{array}\right.$$  

and therefore the oracle $O$ must transform the state according to:  

$$\lvert 00\rangle \rightarrow \left( -1\right) ^{f\left( 00\right) }\left\vert 00\right\rangle =\left\vert 00\right\rangle ,$$  
$$\lvert 01\rangle \rightarrow \left( -1\right) ^{f\left( 01\right) }\left\vert 01\right\rangle =\left\vert 01\right\rangle ,$$  
$$\lvert 10\rangle \rightarrow \left( -1\right) ^{f\left( 10\right) }\left\vert 10\right\rangle = - \left\vert 10\right\rangle ,$$  
$$\lvert 11\rangle \rightarrow \left( -1\right) ^{f\left( 11\right) }\left\vert 11\right\rangle =\left\vert 11\right\rangle ,$$  

* **𝑁=8  example:** If marked item is  𝛽=2  in decimal, then the function  𝑓  is given by  

$$f\left( x\right) =\left\{ \begin{array}{ll} 1 & \text {if }  x=010 ,\\ 0 & \text {otherwise,}  \end{array}\right.$$  

and therefore the oracle  𝑂  must transform according to:  

$$\left\vert 010\right\rangle \rightarrow \left( -1\right) ^{f\left( 010\right) }\left\vert 010\right\rangle =-\left\vert 010\right\rangle$$  

and 

$$\left\vert x\right\rangle \rightarrow \left( -1\right) ^{f\left( x\right) }\left\vert x\right\rangle =\left\vert x\right\rangle \text {, for }x\neq 010.$$  

> **QUESTION: For a list of N=512 items, how many qubits would you need to implement Grover's algorithm?  **
* 9 qubits  
* 512 qubits  
* $2^{512}$ qubits  
* $2\cdot 512$ qubits  

**Now let's go over the algorithm step by step **

## <center>1. Grover's Algorithm: Initialization</center>

**State preparation:**  
At the beginning of the protocol, there is no information about where the marked item is. Thus, it is convenient to start with an equal superposition state

$$\left\vert \psi\right\rangle =\frac{1}{\sqrt{N}}\sum _{x=0}^{N-1}\left\vert x\right\rangle ,$$
 
such that each basis state $\left\vert x\right\rangle $ will have the same probability of being measured:

$$\frac{1}{N}=\frac{1}{2^{n}}.$$
 
Even though we don't know which item is the marked item at the beginning of the algorithm, we can break the equal superposition state into a sum over the marked items and a sum over all other items,  
$$\left\vert \psi\right\rangle =\frac{1}{\sqrt{N}}\sum _{x=0}^{N-1}\left\vert x\right\rangle$$  
$$= \frac{1}{\sqrt{N}}\sum _{x:f(x)=0}\left\vert x\right\rangle +\frac{1}{\sqrt{N}}\sum _{x:f(x)=1}\left\vert x\right\rangle ,$$  
$$=  \underbrace{\frac{1}{\sqrt{N}}\sum _{x:f(x)=0}\left\vert x\right\rangle }_{\text {Non-solution state}}+\underbrace{\frac{1}{\sqrt{N}}\left\vert \beta\right\rangle }_{\text {Solution state}}.$$

To understand the geometrical interpretation of the algorithm, it is convenient to define the normalized quantum state

$$\left\vert \alpha\right\rangle \equiv \frac{1}{\sqrt{\frac{N-1}{N}}}\left( \frac{1}{\sqrt{N}}\sum _{x:f\left( x\right) =0}\left\vert x\right\rangle \right) ,$$  

such that the equal superposition state can be written as the sum

$$\left\vert \psi\right\rangle =\underbrace{\sqrt{\frac{N-1}{N}}\left\vert \alpha\right\rangle }_{\text {Non-solution state}} +\underbrace{\frac{1}{\sqrt{N}}\left\vert \beta\right\rangle }_{\text {Solution state}},$$  

or, equivalently,

$$\left\vert \psi\right\rangle =\cos \left(\frac{\theta }{2}\right)\left\vert \alpha\right\rangle +\sin \left(\frac{\theta }{2}\right)\left\vert \beta\right\rangle ,$$  

where we have defined the angle $\theta$ such that

$$ \cos \left(\frac{\theta }{2}\right) =  \sqrt{\frac{N-1}{N}},$$  
$$ \sin \left(\frac{\theta }{2}\right) =  \frac{1}{\sqrt{N}},$$  

The figure below shows a graphical representation of quantum states $\left\vert \psi\right\rangle$, $\left\vert \beta\right\rangle$, and $\left\vert \alpha\right\rangle$ as vectors in a two-dimensional space. The projection of the superposition state $\left\vert \psi\right\rangle$ onto the solution state $\left\vert \psi\right\rangle$ is equal to $1/\sqrt{N}$, and the projection of $\left\vert \psi\right\rangle$ on $\left\vert \alpha\right\rangle$ is equal to $\sqrt{(N-1)/N}$.

<img src='images/Initialization_N3b.png' width=500 >  


**Two-qubit Initialization:**  
To implement Grover's algorithm for a list of $N=2^n=4$ elements, the compiled algorithm requires two qubits, $n=2$. To prepare the two qubits in an equal superposition state $\left\vert \psi\right\rangle$, a Hadamard gate is applied to each qubit,

$$\lvert 0\rangle\lvert 0\rangle = \frac{\left\vert 0\right\rangle +\left\vert 1\right\rangle }{\sqrt{2}}\frac{\left\vert 0\right\rangle +\left\vert 1\right\rangle }{\sqrt{2}},$$  
$$=  \frac{\left\vert 0\right\rangle \left\vert 0\right\rangle +\left\vert 0\right\rangle \left\vert 1\right\rangle +\left\vert 1\right\rangle \left\vert 0\right\rangle +\left\vert 1\right\rangle \left\vert 1\right\rangle }{2}.$$


If we assume that the marked item is given by $\beta=3$ (decimal representation), then it is convenient to write the two-qubit equal superposition state as

$$\left\vert \psi\right\rangle =\underbrace{\frac{\left\vert 0\right\rangle \left\vert 0\right\rangle +\left\vert 0\right\rangle \left\vert 1\right\rangle +\left\vert 1\right\rangle \left\vert 0\right\rangle }{2}}_{\text {Non-solution state }}+\underbrace{\frac{\left\vert 1\right\rangle \left\vert 1\right\rangle }{2}}_{\text {Solution state}},$$  
 
or as a superposition of the non-solution and solution states

$$\lvert \psi\rangle= \frac{\sqrt{3}}{2}\left\vert \alpha\right\rangle +\frac{1}{2}\left\vert \beta\right\rangle ,$$  
$$ = \displaystyle \cos (30^{\circ})\left\vert \alpha\right\rangle +\sin (30^{\circ})\left\vert \beta\right\rangle ,$$

where the non-solution state is given by

$$\left\vert \alpha\right\rangle =\frac{1}{\sqrt{3}}\left( \left\vert 0\right\rangle \left\vert 0\right\rangle +\left\vert 0\right\rangle \left\vert 1\right\rangle +\left\vert 1\right\rangle \left\vert 0\right\rangle \right) ,$$  
 
and the solution state by

$$\left\vert \beta\right\rangle =\left\vert 1\right\rangle \left\vert 1\right\rangle .$$
 
The figure below shows the graphical representation of the two-qubit equal superposition state  ∣∣𝜓⟩ . The projection of $\left\vert \psi\right\rangle$ on $\left\vert \alpha\right\rangle$ and $\left\vert \beta\right\rangle$ are $\sqrt{3}/2$  and $1/2$ respectively.  

<img src='images/Initialization_N2d.png' width=200 >  

> **TODO: Two-qubit Equal Superposition State  
 Pseudocode that generates the equal superposition state for two-qubits:  **
```python
Apply a Hadamard gate on the first qubit
Apply a Hadamard gate on the second qubit
Save the measurement result of the first qubit on the first bit
Save the measurement result of the second qubit on the second bit
```

> **TODO: Three-qubit Equal Superposition State  
pseudocode that generates the equal superposition state for three-qubits:**
```python
Apply a Hadamard gate on the first qubit
Apply a Hadamard gate on the second qubit
Apply a Hadamard gate on the third qubit
Save the measurement result of the first qubit on the first bit
Save the measurement result of the second qubit on the second bit
Save the measurement result of the third qubit on the third bit
```

### Solution: 
Let's do for the case of three. We chose pyquil to do so. Feel free to code in qiskit or any other language you like.

In [6]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
from pyquil.api import get_qc
qc = get_qc("2q-qvm")

In [19]:
# Two-qubit Equal Superposition State

## define number of classical registers (bits)
cr = 2

# initialize state (in |000>)
two_qubit_superposition = Program()
# declare classical registers to store measurement results in
#ro = two_qubit_superposition.declare('ro', 'BIT', cr)
two_qubit_superposition += H(0)
two_qubit_superposition += H(1)
#two_qubit_superposition += MEASURE(0, ro[0])
#two_qubit_superposition += MEASURE(1, ro[1])

print('two qubit superposition circuit\n\n', two_qubit_superposition)

#two_qubit_superposition.wrap_in_numshots_loop(10)
#compiled_program = qc.compile(two_qubit_superposition)
#results = qc.run(compiled_program)

# let's see the outcomes of measurements
# as the measured state is an equal superposition of basis states
# we should see randome outcomes
#print(results)

two qubit superposition circuit

 H 0
H 1



## <center>2. Grover's Algorithm: Oracle O</center>

**Oracle operation:**  
The role of the oracle, $O$, is to encode the function  $f(x)$ in the quantum system and mark the solution. The operation transforms each state of the computational basis as  

$$\left\vert x\right\rangle \to \left( -1\right)^{f\left( x\right) }\left\vert x\right\rangle .$$
 
In the first iteration of the protocol, this operation transforms the equal superposition state according to:

$$O\left\vert \psi \right\rangle =  \cos \left(\frac{\theta }{2}\right)O\left\vert \alpha \right\rangle +\sin \left(\frac{\theta }{2}\right)O\left\vert \beta\right\rangle ,$$  
$$ = \cos \left(\frac{\theta }{2}\right)\left\vert \alpha\right\rangle -\sin \left(\frac{\theta }{2}\right)\left\vert \beta\right\rangle .$$  

The figure below shows the graphical representation of the equal superposition state after the Oracle operator, $O\left\vert \psi \right\rangle$. The amplitude of $\left\vert \psi \right\rangle$ in front of $O\left\vert \alpha \right\rangle$ remains the same as before. However, the amplitude in front of $\left\vert \beta \right\rangle$ acquires a negative sign. The oracle operation serves to reflect the state $\left\vert \psi \right\rangle$ about the non-solution state $\left\vert \alpha \right\rangle$.

<img src='images/OracleN.png' width=400 >

**N=4 item list**  
* **Example 1:** If the marked item is $\beta=3$ (decimal representation), then the function $f$ is given by

$$f\left( x\right) =\left\{ \begin{array}{cc} 1, & \text {if }x=11, \\ 0, & \text {otherwise,}\end{array}\right.$$  
and therefore the oracle $O$ must transform according to:

$$\left\vert \psi \right\rangle \rightarrow \cos \left( 30^{\circ}\right) \left\vert \alpha \right\rangle -\sin \left( 30^{\circ}\right) \left\vert \beta\right\rangle ,$$
 
where the non-solution state is  

$$ \left\vert \alpha \right\rangle =\frac{1}{\sqrt{3}}\left( \left\vert 0\right\rangle \left\vert 0\right\rangle +\left\vert 0\right\rangle \left\vert 1\right\rangle +\left\vert 1\right\rangle \left\vert 0\right\rangle \right) ,$$  
 
and the solution state is

$$\left\vert \beta\right\rangle =\left\vert 1\right\rangle \left\vert 1\right\rangle .$$  
 
In particular, the Oracle must transform each element of the two-qubit basis according to:

$$\lvert 0\rangle\lvert 0\rangle \rightarrow  
\left( -1\right) ^{f\left( 00\right) }
\left\vert 0\right\rangle \left\vert 0\right\rangle =\left\vert 0\right\rangle \left\vert 0\right\rangle ,$$  
$$\lvert 0\rangle\lvert 1\rangle \rightarrow  
\left( -1\right) ^{f\left( 01\right) }
\left\vert 0\right\rangle \left\vert 1\right\rangle =\left\vert 0\right\rangle \left\vert 1\right\rangle ,$$  
$$\lvert 1\rangle\lvert 0\rangle \rightarrow  
\left( -1\right) ^{f\left( 10\right) }
\left\vert 1\right\rangle \left\vert 0\right\rangle =\left\vert 1\right\rangle \left\vert 0\right\rangle ,$$  
$$\lvert 1\rangle\lvert 1\rangle \rightarrow  
\left( -1\right) ^{f\left( 11\right) }
\left\vert 1\right\rangle \left\vert 1\right\rangle =-\left\vert 1\right\rangle \left\vert 1\right\rangle ,$$  
The figure below shows a graphical representation of the non-solution, solution, and reflected states.  

<img src='images/Oracle2b.png' width=200 >  

The quantum circuit that implements the Oracle for this marked item is basically a Controlled Z operator:  

$$CZ = \begin{bmatrix} 
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & -1 
\end{bmatrix}$$

The table below shows the evolution of the two-qubit basis states. You can see that the only basis state that acquires a negative sign is the solution state. 


|Basis State|After the $CZ$  gate|
|---|---|
|$\lvert 0\rangle \lvert 0\rangle$|$\lvert 0\rangle \lvert 0\rangle$|
|$\lvert 0\rangle \lvert 1\rangle$|$\lvert 0\rangle \lvert 1\rangle$|
|$\lvert 1\rangle \lvert 0\rangle$|$\lvert 1\rangle \lvert 0\rangle$|
|$\lvert 1\rangle \lvert 1\rangle$|$-\lvert 1\rangle \lvert 1\rangle$|


> **TODO: N=4: Oracle Operator $\beta=3$  
For a list of $N=4$ items, write code that generates the Oracle operator when the marked item is $\beta=3$ (in decimal representation, $\beta=11$ in binary representation). Don't forget to measure your two qubits. Compare your results with the table above using $\lvert 0\rangle\lvert 0\rangle$.**



In [3]:
cr = 2

# initialize state (in |000>)
oracle = Program()
# declare classical registers to store measurement results in
oracle += CZ(0, 1)
print(oracle)



CZ 0 1



## <center>3. Grover's Algorithm: Grover Operator</center>
**Reflection  𝑊 :**  
The third step in Grover's algorithm is given by the implementation of the second reflection, which we will call  𝑊 . Let's recall the state after the first implementation of the Oracle Operator,

$$O\left\vert \psi \right\rangle =\cos \left( \frac{\theta }{2}\right) \left\vert \alpha \right\rangle -\sin \left( \frac{\theta }{2}\right) \left\vert \beta\right\rangle.$$
 
The $W$ operator then corresponds to a reflection about the equal superposition state $\lvert\psi\rangle$. Since this operator does not depend on the function  $f(x)$, and, rather depends only on the superposition state, $W$ is the same for any marked item $\beta$. Let's write its explicitly again for reference:  

$$W = H^{\otimes n}(2\lvert 0\rangle \langle 0 \rvert - I)H^{\otimes n} = 2\lvert \psi\rangle \langle \psi \rvert - I. \hspace{2cm} (1)$$  


**Grover Operator**  
We define the Grover operator as 

$$G=WO,$$
that is, the application of the Oracle operator followed by the $W$ reflection.

$$G\left\vert \psi \right\rangle =\cos \left( \frac{3\theta }{2}\right) \left\vert \alpha \right\rangle +\sin \left( \frac{3\theta }{2}\right) \left\vert \beta\right\rangle$$  
 
<img src='images/NGroverc.png' width=400 >  

The state after the second Grover iteration is:

$$G^{2}\left\vert \psi \right\rangle =\cos \left( \frac{5\theta }{2}\right) \left\vert \alpha \right\rangle +\sin \left( \frac{5\theta }{2}\right) \left\vert \beta\right\rangle$$
 
<img src='images/NGrover2b.png' width=500 >  

In general, the state after the k-th Grover iteration will be given by

$$G^{k}\left\vert \psi \right\rangle =\cos \left( \frac{2k+1}{2}\theta \right) \left\vert \alpha \right\rangle +\sin \left( \frac{2k+1}{2}\theta \right) \left\vert \beta\right\rangle$$
 
**N=4 items: Marked item $\beta=11$**  
As you learned in the previous section, the equal superposition state for a list of 4 items is given by:

$$\left\vert \psi \right\rangle =\cos \left( 30^{\circ}\right) \left\vert \alpha \right\rangle +\sin \left( 30^{\circ}\right) \left\vert \beta\right\rangle ,$$
 
where the non-solution and solution states are respectively given by

$$\lvert\alpha\rangle = \frac{\left\vert 0\right\rangle \left\vert 0\right\rangle +\left\vert 0\right\rangle \left\vert 1\right\rangle +\left\vert 1\right\rangle \left\vert 0\right\rangle }{\sqrt{3}},$$  
$$\lvert\beta\rangle = \left\vert 1\right\rangle \left\vert 1\right\rangle .$$

After the Oracle operator for the item $\beta=11$ is applied, the equal superposition state $\lvert\psi\rangle$ is reflected about the non-solution state $\lvert\alpha\rangle$

$$O\left\vert \psi \right\rangle =\cos \left( 30^{\circ}\right) \left\vert \alpha \right\rangle -\sin \left( 30^{\circ}\right) \left\vert \beta\right\rangle .$$
 
The second reflection is applied, and this reflects the state  $O\lvert\psi\rangle$  about the equal superposition state $\lvert\psi\rangle$,

$$WO\left\vert \psi \right\rangle =\cos \left( 90^{\circ}\right) \left\vert \alpha \right\rangle +\sin \left( 90^{\circ}\right) \left\vert \beta\right\rangle .$$
 
How many times should we apply the Grover Operator? For a list of $N=4$ items and one marked item $\beta$, the number of necessary repetitions $R$ required to find $W$ is bounded above by:

$$R \leq \frac{\pi }{4}\sqrt{4} \leq 1.5.$$  

Since the upper bound is $\approx 1.5$, we know that the optimal number of iterations to find the marked item is equal to one. The state after the first Grover iteration, $G=WO$, is given by

$$G\left\vert \psi \right\rangle =\cos \left( 90^{\circ}\right) \left\vert \alpha \right\rangle +\sin \left( 90^{\circ}\right) \left\vert \beta\right\rangle .$$  
 
The figure below illustrates the final state after the first Grover iteration. You can see that the **final state matches the solution state**. 

<img src='images/N4w3Groverc.png' width=200 >  

From the figure below, you can see that applying the Grover operator a second time, $G^2=(WO)^2$ , does not improve the performance of the algorithm. After a second iteration, the resulting state **moves away** from the solution state,

$$G^{2}\left\vert \psi \right\rangle =\cos \left( 150\right) \left\vert \alpha \right\rangle +\sin \left( 150\right) \left\vert \beta\right\rangle .$$
 
<img src='images/N4w3Grover2b.png' width=600 >  

> **TODO: Implement the reflection $W$ (eq (1)) for a list of $N=4$ items. Write code that generates the quantum circuit.**

In [9]:
reflection = Program()
reflection += H(0)
reflection += H(1)
reflection += X(0)
reflection += X(1)
reflection += CZ(0, 1)
reflection += X(0)
reflection += X(1)
reflection += H(0)
reflection += H(1)
print(reflection)

H 0
H 1
X 0
X 1
CZ 0 1
X 0
X 1
H 0
H 1



### <center> Grover's Algorithm for different solution cases </center>
The quantum circuit to implement Grover's algorithm for a list of $N=4$ items with $\beta=00$

In [36]:
initialization = two_qubit_superposition
grover00 = Program()
ro = two_qubit_superposition.declare('ro', 'BIT', cr)

# oracle
oracle00 = Program()
oracle00 += S(0)
oracle00 += S(1)
oracle00 += CZ(0, 1)
oracle00 += S(0)
oracle00 += S(1)

# reflection
grover00 += oracle00
grover00 += reflection
grover00 = initialization + grover00

grover00 += MEASURE(0, ro[0])
grover00 += MEASURE(1, ro[1])

grover00.wrap_in_numshots_loop(10)
compiled_program = qc.compile(grover00)
results00 = qc.run(compiled_program)


print(grover00)
print(results00)

H 0
H 1
DECLARE ro BIT[2]
S 0
S 1
CZ 0 1
S 0
S 1
H 0
H 1
X 0
X 1
CZ 0 1
X 0
X 1
H 0
H 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


The quantum circuit to implement Grover's algorithm for a list of $N=4$ items with $\beta=01$

In [38]:
grover01 = Program()

# oracle
oracle01 = Program()
oracle01 += S(1)
oracle01 += CZ(0, 1)
oracle01 += S(1)

# reflection
grover01 += oracle01
grover01 += reflection
grover01 = initialization + grover01

grover01 += MEASURE(0, ro[0])
grover01 += MEASURE(1, ro[1])

grover01.wrap_in_numshots_loop(10)
compiled_program = qc.compile(grover01)
results01 = qc.run(compiled_program)


print(grover01)
print(results01)

H 0
H 1
DECLARE ro BIT[2]
S 1
CZ 0 1
S 1
H 0
H 1
X 0
X 1
CZ 0 1
X 0
X 1
H 0
H 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


The quantum circuit to implement Grover's algorithm for a list of $N=4$ items with $\beta=10$

In [39]:
grover10 = Program()

# oracle
oracle10 = Program()
oracle10 += S(0)
oracle10 += CZ(0, 1)
oracle10 += S(0)

# reflection
grover10 += oracle10 
grover10 += reflection
grover10 = initialization + grover10

grover10 += MEASURE(0, ro[0])
grover10 += MEASURE(1, ro[1])

grover10.wrap_in_numshots_loop(10)
compiled_program = qc.compile(grover10)
results10 = qc.run(compiled_program)


print(grover10)
print(results10)

H 0
H 1
DECLARE ro BIT[2]
S 0
CZ 0 1
S 0
H 0
H 1
X 0
X 1
CZ 0 1
X 0
X 1
H 0
H 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


And finally, the quantum circuit to implement Grover's algorithm for a list of $N=4$ items with $\beta=11$

In [40]:
grover11 = Program()

# oracle
oracle11 = CZ(0, 1)

# reflection
grover11 += oracle11
grover11 += reflection

grover11 = initialization + grover11
grover11 += MEASURE(0, ro[0])
grover11 += MEASURE(1, ro[1])

grover11.wrap_in_numshots_loop(10)
compiled_program = qc.compile(grover11)
results11 = qc.run(compiled_program)


print(grover11)
print(results11)

H 0
H 1
DECLARE ro BIT[2]
CZ 0 1
H 0
H 1
X 0
X 1
CZ 0 1
X 0
X 1
H 0
H 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]

[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]


OK, this was a warmup toy case, and was intended to give the flavor of how to think about writing code to generate parts of the algorithm. See the next tutorial for the complete algorithm for a 3-SAT problem.